In [170]:
import selenium
from selenium import webdriver
import time,re,random,sys
import pandas as pd
from bs4 import BeautifulSoup 
#import requests
#from fake_useragent import UserAgent
#import urllib
# import tools
from selenium.common.exceptions import NoSuchElementException,TimeoutException,WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [208]:
def deal_comments_infos_fromurlist(path):
    
    def get_comments1(ps):    #tmall
        comments=[]
        for comment in re.findall('<div class="tm-rate-content"><div class="tm-rate-fulltxt">(.*?)</div>',ps):
            if 'span' not in comment:
                comments.append(comment)
        return pd.DataFrame({
                'comment':comments
        })
    
    def get_comments2(ps):     #taobao
        soup = BeautifulSoup(driver.page_source, 'lxml')
        a = [i.text.strip() for i in soup.findAll('div',{'class':'J_KgRate_ReviewContent tb-tbcr-content '})]
        comments = []
        for j,k in enumerate(a):
            if '[追加评论]' in k:
                continue
            if j != len(a)-1:
                if '评价方未及时' in k or '此用户没有填写评价' in k:
                    if '评价方未及时' not in a[j+1] and '此用户没有填写评价' not in a[j+1]:
                        comments.append(a[j+1].strip('[追加评论]\n          \n          '))
                    else:
                        continue
                else:
                    comments.append(k)
            else:
                comments.append(k)
        return pd.DataFrame({
                'comment':comments
        })
    
    def title_process(title):
        xtitle = title
        xlist = ['\\','/',':','*','"','<','>','|','?',' ']
        for i in xlist:
            if i in xtitle:
                 xtitle = title.split(i)[0]
        return(xtitle)
        
    
    def tmallmethod(url):
        
        try:
            driver.get(url)
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.ID, "J_TabBarBox"))  # 判断页面是否初步加载成功的标记
            )
        except TimeoutException:
            print('宝贝链接未加载成功')

        soup = BeautifulSoup(driver.page_source, 'lxml')
        title = title_process(soup.findAll(name='h1',attrs={'data-spm':"1000983"})[0].text.strip())

        try:
            # 页面上拉600，看到TabBarBox
            js = "window.scrollTo(0,600)"
            driver.execute_script(js)
        except WebDriverException:
            print('上拉寻找评论区时出现问题')

        time.sleep(random.uniform(3,4))
        # 点击累计评论TAB
        driver.find_element_by_xpath('//a[@href="#J_Reviews"]').click()
        time.sleep(random.uniform(1,2))
        print('成功点击累计评论TAB')

        while True:
            try:
                # rate-grid的正则表达式
                driver.find_element_by_xpath('//*[@id="J_Reviews"]/div/div[6]')
                print('已经成功加载到评论区信息')
                break
            except NoSuchElementException:
                print('评论信息中元素还未加载')
                continue


        allcmts = pd.DataFrame()

        count = 2
        while count<10:

            cmts = get_comments1(driver.page_source)
            allcmts = pd.concat([allcmts,cmts],axis=0,ignore_index=True)

            try:
                js = "window.scrollTo(0,2500)"
                driver.execute_script(js)
                time.sleep(random.uniform(1,2))
                # 点击下一页
                nextpage = driver.find_element_by_css_selector('#J_Reviews > div > div.rate-page > div > a:last-child')
                driver.execute_script('arguments[0].click()',nextpage)
                print('\r已成功点击第' + str(count) +'页',end='')

            except NoSuchElementException:
                print('找不到翻页按钮,结束查询--(%s)' % title)
                break

            count += 1
            time.sleep(random.uniform(1,2))

        today = str(time.strftime('%Y-%m-%d',time.localtime(time.time())))
        allcmts.to_csv('%s-%s.csv'%(today,title),index=False,header=False,encoding='utf_8_sig')
        
    def taobaomethod(url):
        try:
            driver.get(url)
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.ID, "J_TabBar"))  # 判断页面是否初步加载成功的标记
            )
        except TimeoutException:
            print('宝贝链接未加载成功')

        soup = BeautifulSoup(driver.page_source, 'lxml')
        title = title_process([i.find('h3')['data-title'] for i in soup.findAll(name='div',attrs={'class':"tb-title"})][0].strip())

        try:
            # 页面上拉600，看到TabBarBox
            js = "window.scrollTo(0,1000)"
            driver.execute_script(js)
        except WebDriverException:
            print('上拉寻找评论区时出现问题')

        time.sleep(random.uniform(3,4))
        # 点击累计评论TAB
        driver.find_element_by_xpath('//ul[@id="J_TabBar"]/li[2]').click()
        print('成功点击累计评论TAB')
        time.sleep(random.uniform(1,2))

        while True:
            try:
                # rate-grid的正则表达式
                driver.find_element_by_xpath('//div[@class="sub-wrap"]')
                print('已经成功加载到评论区信息')
                break
            except NoSuchElementException:
                print('评论信息中元素还未加载')
                continue

        allcmts = pd.DataFrame()

        count = 2
        while count<100:

            cmts = get_comments2(driver.page_source)
            allcmts = pd.concat([allcmts,cmts],axis=0,ignore_index=True)

            try:
                js = "window.scrollTo(0,4000)"
                driver.execute_script(js)
                time.sleep(random.uniform(1,2))
                # 点击下一页
                nextpage = driver.find_element_by_xpath('//li[@class="pg-next"]')
                driver.execute_script('arguments[0].click()',nextpage)
                print('\r已成功点击第' + str(count) +'页',end='')

            except NoSuchElementException:
                print('找不到翻页按钮,结束查询--(%s)' % title)
                break

            count += 1
            time.sleep(random.uniform(1,2))

        today = str(time.strftime('%Y-%m-%d',time.localtime(time.time())))
        allcmts.to_csv('%s-%s.csv'%(today,title),index=False,header=False,encoding='utf_8_sig')
            
    set = pd.read_csv(path,header=None) 
    urlist = [ i[0] for i in set.values.tolist()]

    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-automation']) 
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    driver = webdriver.Chrome(options = options)
    driver.set_window_size(1000,700)
    timeout = 30
        
    for url in urlist:
        if not url.startswith('http'):
            url = 'https:' + url
        print('开始寻找评论from:' + url)
        if url.startswith('https://item.taobao.com/'):
            taobaomethod(url)
        elif url.startswith('https://detail.tmall.com/'):
            tmallmethod(url)
        else:
            print('网址错误%s'%url)
            break
        
    driver.close()

In [ ]:
deal_comments_infos_fromurlist('123.csv')

In [129]:
# 确认你的网页池
set = pd.read_csv(path,header=None) 
urlist = [ i[0] for i in set.values.tolist()]